The defensive linemen are the primary line of defense against the enemy. Ultimate job of offensive line is to give enough time and protection to the quarterback, so that the quarterback can make a successful pass. But it is pretty difficult to evaluate them and there haven't been effective matrices made that can evaluate them with the data available. Here i am presenting a few ideas that can be potentially used to create such a matric.

## How to evaluate the linemen?

Keeping the job of defensive linemen in mind, in simpler terms, it can be said that, if the offensive linemen are doing a good job, they will provide more time to quarterback to make a successful throw. But it is not always true, since it depends on more dynamics than just that. But, A quarterback will be in less pressure if the offensive linemen are able to keep the defensive linemen at bay. And, also stop the blitzing players that are coming towards quarterback. 

A good way to look at this scenario is to consider that, the quarterback has a fixed amount of area available to him, considering the enemy playes around him. The more the enemies come closer, the lesser the area the quarterback will have and more pressure the quarterback will feel. Also, if the enemy player is able to come past the offense,there will be a potential for a sack, the quarterback will have significantly less time in that situation. 

We will be looking at how we can visualize following data and extract information about them from each play in order to understand how the offensive linemen performed.
1. Area occupation by the quarterback
2. The enemies that can potentially sack the quarterback

First we will prepare the data.

**Importing necessary libraries**

In [1]:
import pandas as pd
import numpy as np
import random
import os
import seaborn as sns
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
from matplotlib.patches import Circle
from scipy.spatial import Voronoi,voronoi_plot_2d, ConvexHull
from shapely.geometry import LineString
from shapely.geometry import Point, Polygon
from shapely.geometry import shape
from IPython import display

**Reading files**

In [2]:
games = pd.read_csv("games.csv")
pffScoutingData = pd.read_csv("pffScoutingData.csv")
players =  pd.read_csv("players.csv")
plays =  pd.read_csv("plays.csv")
week1 =  pd.read_csv("week1.csv")
week2 =  pd.read_csv("week2.csv")
week3 =  pd.read_csv("week3.csv")
week4 =  pd.read_csv("week4.csv")
week5 =  pd.read_csv("week5.csv")
week6 =  pd.read_csv("week6.csv")
week7 =  pd.read_csv("week7.csv")
week8 =  pd.read_csv("week8.csv")

In [3]:
# games = pd.read_csv("../input/nfl-big-data-bowl-2023/games.csv")
# pffScoutingData = pd.read_csv("../input/nfl-big-data-bowl-2023/pffScoutingData.csv")
# players =  pd.read_csv("../input/nfl-big-data-bowl-2023/players.csv")
# plays =  pd.read_csv("../input/nfl-big-data-bowl-2023/plays.csv")
# week1 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week1.csv")
# week2 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week2.csv")
# week3 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week3.csv")
# week4 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week4.csv")
# week5 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week5.csv")
# week6 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week6.csv")
# week7 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week7.csv")
# week8 =  pd.read_csv("../input/nfl-big-data-bowl-2023/week8.csv")

In [4]:
def voronoi_area_percentage_with_common_edge(circle_radius, xy, qbIndex):
    voronoi_areas=[]
    common_edge = []
    circle = Point(xy[qbIndex][0],xy[qbIndex][1]).buffer(circle_radius)
    n_points = xy.shape[0]
    xy1 = xy.copy()
    xy1[:,1] = - xy[:,1]
    xy2 = xy.copy()
    xy2[:,1] = 320/3 - xy[:,1]
    xy3 = xy.copy()
    xy3[:,0] = - xy[:,0]
    xy4 = xy.copy()
    xy4[:,0] = 240 - xy[:,0]
    xy = np.concatenate((xy, xy1, xy2, xy3, xy4), axis=0)
    
    vor = Voronoi(xy)
    
    
    
    points= vor.regions[vor.point_region[qbIndex]] # list of vertex indices that the quarterback has. 
    # The points that have even one common vertex are the ones that are in direct contact with QB
    
    for r in range(n_points):
        region = vor.regions[vor.point_region[r]]
        if not -1 in region:
            polygonShapely = Polygon([vor.vertices[i] for i in region])
            if polygonShapely.intersects(circle):
                voronoi_areas.append(round((polygonShapely.intersection(circle).area/circle.area)*100,1))
                if any(x in vor.regions[vor.point_region[r]] for x in points):
                    common_edge.append(1)
                else:
                    common_edge.append(0)
                
            else:
                voronoi_areas.append(0)
                common_edge.append(0)
        else:
            voronoi_areas.append(0)
            common_edge.append(0)
            
    return voronoi_areas, common_edge

In [6]:
# Combining all week data
week = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])
week['nflId'] = week['nflId'].fillna(0).astype(np.int64)
week[['gameId','playId', 'nflId']] = week[['gameId','playId', 'nflId']].applymap(str)
plays[['gameId','playId']] = plays[['gameId','playId']].applymap(str)
players[['nflId']] =players[['nflId']] .applymap(str)

# Creating uniquePlay Columns
week['uniquePlay'] = week['gameId'] + week['playId'] 
plays['uniquePlay'] = plays['gameId'] + plays['playId']

week = week[((week.groupby('uniquePlay')['event'].transform(lambda x: x.eq('ball_snap').any())) & (week.groupby('uniquePlay')['event'].transform(lambda x: x.eq('pass_forward').any())))]

all_uniquePlays = week['uniquePlay'].unique()

In [7]:
global selected_play
selected_play =all_uniquePlays[random.choice(range(1,len(all_uniquePlays)+1))]


weekSubset= week.loc[week['uniquePlay'] == selected_play, :]


# Merge week with plays and players
weekSubset= pd.merge(weekSubset,plays[['uniquePlay','possessionTeam', 'defensiveTeam','passResult']], on='uniquePlay', how="left")
weekSubset= pd.merge(weekSubset,players[['nflId','officialPosition']], on='nflId', how="left")
weekSubset['officialPosition'] = weekSubset['officialPosition'].fillna("BALL")

# Create OffenceFlag, QBFlag, OffLineFlag for identification of player type
weekSubset['OffenceFlag'] = (weekSubset['possessionTeam'] == weekSubset['team']).astype(int)
weekSubset['QBFlag'] = (weekSubset['officialPosition'] == "QB").astype(int)
weekSubset['OffLineFlag'] = weekSubset['officialPosition'].isin(['C', 'G', 'T']).astype(int)

# Finding and keeping only the data between ball snap and pass forward
weekSubset["ball_snap_frame"] = weekSubset[(weekSubset['event'] == 'ball_snap')]['frameId']
weekSubset["pass_forward_frame"] = weekSubset[ (weekSubset['event'] == 'pass_forward')]['frameId']
weekSubset["ball_snap_frame"] = weekSubset["ball_snap_frame"].fillna(0).astype('int64')
weekSubset["pass_forward_frame"] = weekSubset["pass_forward_frame"].fillna(0).astype('int64')
weekSubset.groupby('uniquePlay')['ball_snap_frame'].transform("max")
weekSubset = weekSubset[weekSubset['frameId']>= weekSubset.groupby('uniquePlay')['ball_snap_frame'].transform("max")]
weekSubset.groupby('uniquePlay')['pass_forward_frame'].transform("max")
weekSubset = weekSubset[weekSubset['frameId']<= weekSubset.groupby('uniquePlay')['pass_forward_frame'].transform("max")]

# Creating voronoi areas and storing data
weekSubset['voronoi_rate']= np.nan
i = selected_play
for frame in range(weekSubset[weekSubset['uniquePlay']==i ]['frameId'].min(),weekSubset[weekSubset['uniquePlay']==i ]['frameId'].max()+1):
    xy = weekSubset[(weekSubset['uniquePlay']== i) & (weekSubset['frameId']== frame) & (weekSubset['team'] != 'football') ]
    qbIndex = np.where(xy['officialPosition']=='QB')[0][0]
    xy= xy.loc[:,'x':'y'].values
    weekSubset.loc[(weekSubset['uniquePlay']== i) & (weekSubset['frameId']== frame) & (weekSubset['team'] != 'football'),['voronoi_rate']], weekSubset.loc[(weekSubset['uniquePlay']== i) & (weekSubset['frameId']== frame) & (weekSubset['team'] != 'football'),'common_edge']  =  voronoi_area_percentage_with_common_edge(5,xy,qbIndex) 
weekSubset.loc[:, 'Offence_voronoi'] = weekSubset['OffenceFlag'].multiply(weekSubset['voronoi_rate'])


# Create a visalisation
figMulti, axes = plt.subplots(2,1)
SinglePlay= weekSubset[(weekSubset.team != 'football')& ((weekSubset.OffenceFlag ==1))]

grouped_frame=SinglePlay[SinglePlay.uniquePlay==selected_play].groupby('frameId')
offence_voronoi=grouped_frame[['voronoi_rate']].sum().reset_index()
offence_voronoi['defence_voronoi_rate'] = 100- offence_voronoi['voronoi_rate']
offence_data = offence_voronoi.voronoi_rate
defence_data = offence_voronoi.defence_voronoi_rate


frame=[]
offence_line=[]
defence_line=[]
frames=offence_voronoi.frameId
axes[1].set_xlim(min(frames), max(frames))
axes[1].set_ylim(0,100)    
figMulti.set_figheight(8)
figMulti.set_figwidth(6)
axes[1].set_title(f"Area occupation by players in 5 yard circle around QB\n\nPlay ID : {selected_play}",fontsize=10)
axes[1].grid()
anim =animation.FuncAnimation(figMulti,lambda i: show_voronoi_with_graph(i,selected_play), range(min(frames), max(frames)+1),interval=100)

#anim.save('animation.gif', fps=10, dpi=200)


# converting to an html5 video
video = anim.to_html5_video()
  
# embedding for the video
html = display.HTML(video)
  
# draw the animation
display.display(html)
plt.close()

In [8]:
frame

[6,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35]

In [10]:
def show_voronoi_with_graph(i, uniqueplay):
    axes[0].cla()
    play1 = weekSubset[(weekSubset['uniquePlay'] == uniqueplay) & (weekSubset['frameId'] == i) & (weekSubset['team'] != 'football')]
    ball1 = weekSubset[(weekSubset['uniquePlay'] == uniqueplay) & (weekSubset['frameId'] == i) & (weekSubset['team'] == 'football')]
    QBx=  np.where(play1['QBFlag']==1)[0][0]
    circle = Point(play1.iloc[QBx]['x'],play1.iloc[QBx]['y']).buffer(5)
    
    xy = play1.loc[:,'x':'y'].values
    n_points = xy.shape[0]
    xy1 = xy.copy()
    xy1[:,1] = - xy[:,1]
    xy2 = xy.copy()
    xy2[:,1] = 320/3 - xy[:,1]
    xy3 = xy.copy()
    xy3[:,0] = - xy[:,0]
    xy4 = xy.copy()
    xy4[:,0] = 240 - xy[:,0]
    xy = np.concatenate((xy, xy1, xy2, xy3, xy4), axis=0)
    offence = play1['OffenceFlag']
    common_edge  = play1['common_edge']
    vor = Voronoi(xy)
    axes[0].axis('equal')
    
    for r in range(n_points):
        region = vor.regions[vor.point_region[r]]
        if not -1 in region:
            polygon = [vor.vertices[i] for i in region]
            if offence.iloc[r]:
                axes[0].fill(*zip(*polygon), c='g', alpha=0.25)
            else:
                if common_edge.iloc[r]:
                    axes[0].fill(*zip(*polygon), c='r', alpha = 0.5)
                else:
                    axes[0].fill(*zip(*polygon), c='r', alpha = 0.25)
                
            polygonShapely = Polygon(polygon)
            
            if polygonShapely.intersects(circle):
                axes[0].annotate(f"{round((polygonShapely.intersection(circle).area/circle.area)*100,1)}%",xy=(xy[r,0],xy[r,1]),xytext =(xy[r,0]+0.2, xy[r,1]+0.2))
    axes[0].scatter(play1.x, play1.y, color='black', s=10)
    #axes[0].plot(ball1.loc[:,'x':'y'].values[:,0], ball1.loc[:,'x':'y'].values[:,1], 'ro', markersize =3)
    axes[0].plot(*circle.exterior.xy, color='blue')
    axes[0].set_xlim(play1.iloc[QBx]['x']-5, play1.iloc[QBx]['x']+5)
    axes[0].set_ylim(play1.iloc[QBx]['y']-5, play1.iloc[QBx]['y']+5)
    axes[0].axis('off')
    
    ######################################################################
    offence_line.append(offence_data[i-min(frames)])
    defence_line.append(defence_data[i-min(frames)])
    frame.append(frames[i-min(frames)])
    axes[1].plot(frame,offence_line,color="green")
    axes[1].plot(frame,defence_line,color="red")
